In [1]:
# First we load our location data and drop any duplicated places within documents
import pandas as pd
import shapely.vectorized
from global_land_mask import globe

import warnings
import time

# Load location data 
places = pd.read_csv('data/clean_places.csv')                                                                                                                                                                                                                                                                                                                                                                                                                              
places = places.drop_duplicates(["doc_id","geonameid"]) 
print(places.shape)
places.head()

(139808, 13)


,word,spans,country_predicted,country_conf,admin1,lat,lon,country_code3,geonameid,place_name,feature_class,feature_code,doc_id
0,Shanghai,"[{'start': 162, 'end': 170}]",CHN,0.963741,Shanghai,31.22222,121.45806,CHN,1796236.0,Shanghai,P,PPLA,4963912
5,Aeromonas,"[{'start': 1153, 'end': 1162}]",NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4729030
6,Thailand,"[{'start': 57, 'end': 65}]",THA,0.948191,NaN,15.50000,101.00000,THA,1605651.0,Kingdom of Thailand,A,PCLI,4695222
7,Bangkok,"[{'start': 302, 'end': 309}]",THA,0.961249,Bangkok,13.75398,100.50144,THA,1609350.0,Bangkok,P,PPLC,4695222
9,Israel,"[{'start': 285, 'end': 291}]",ISR,0.948191,NaN,31.50000,34.75000,ISR,294640.0,State of Israel,A,PCLI,4762134


In [2]:
df = pd.read_feather("data/included_studies.feather")
df.head()

,id,content,title,0 - relevant,2 - 1.02. Changes in temperature,2 - 1.03. Seasonal change,2 - 1.04. Changes in precipitation,2 - 1.06. Climate change (unspecified),2 - 1.07. Other meteorological variables,3 - 2.01. Food security,...,4 - 2.04. Extreme event attribution,4 - 2.05. Scenarios,5 - 4.01. Floods and drought,5 - 4.02. Heatwaves,5 - 4.03. Wildfires,5 - 4.04. Other extreme events,5 - 4.05. Extreme cold,6 - 5.01. Pollution,6 - 5.03. Reduced agricultural & aquaculture productivity,6 - 5.04. Reduced labour and physical capacity
0,20881,"Climate change will result in more intense, mo...",Projections of heat waves with high impact on ...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
1,27158,"Temperature, a key climate change indicator, i...",Apparent Temperature and Cause-Specific Mortal...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,30042,Climate change has led to significant rise of ...,Climate Change and the Emergent Epidemic of CK...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,30687,BACKGROUND: The periods of fetal and child dev...,Children are likely to suffer most from our fo...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,35987,Background Extreme precipitation events are in...,Autochthonous Chikungunya Transmission and Ext...,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
import numpy as np
def generate_grid_df(degrees):
    '''
    Generate a dataframe with a grid of of cells degrees x degrees
    '''
    LON = np.linspace(-180+degrees*0.5,180-degrees*0.5,int(360/degrees))
    LAT = np.linspace(-90+degrees*0.5,90-degrees*0.5,int(180/degrees))
    lon_df, lat_df = np.meshgrid(LON,LAT)

    return pd.DataFrame({"LAT": lat_df.ravel(), "LON": lon_df.ravel()})
    
grid_df = generate_grid_df(2.5)
grid_df.head()

,LAT,LON
0,-88.75,-178.75
1,-88.75,-176.25
2,-88.75,-173.75
3,-88.75,-171.25
4,-88.75,-168.75


In [4]:
import cartopy.io.shapereader as shpreader
import geopandas
# First we define a list of the shapefile definitions we want
shpfiles = [
    dict(resolution='50m', category='cultural', name='admin_0_countries'),
    dict(resolution='10m', category='cultural', name='admin_1_states_provinces'),
    dict(resolution='10m', category='physical', name='geography_regions_polys'),
    dict(resolution='10m', category='physical', name='geography_marine_polys')
    #"data/gadm36_1.shp"
]

/home/max/.local/lib/python3.10/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(
/tmp/ipykernel_375273/1055013544.py:2: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.h

In [5]:
# We'll start an empty dataframe to store our shapefile-grid matches
shp_grid_df = pd.DataFrame()

# Now we download the shapefiles and combine into one large geopandas dataframe
shp_df  = None
for shpfilename in shpfiles:
    print(f"reading shapefile {shpfilename}")
    if shp_df is None:
        shp_df = geopandas.read_file(shpreader.natural_earth(**shpfilename))
    else:
        shp_df = shp_df.merge(geopandas.read_file(shpreader.natural_earth(**shpfilename)),how="outer")
    time.sleep(10) # Wait a bit before downloading the next shapefile so we do not make too many requests too quickly

reading shapefile {'resolution': '50m', 'category': 'cultural', 'name': 'admin_0_countries'}
reading shapefile {'resolution': '10m', 'category': 'cultural', 'name': 'admin_1_states_provinces'}
reading shapefile {'resolution': '10m', 'category': 'physical', 'name': 'geography_regions_polys'}
reading shapefile {'resolution': '10m', 'category': 'physical', 'name': 'geography_marine_polys'}


In [6]:




degrees = 2.5

# We are going to store our shapefile-gridcell index matches here
shp_grid = []

# This is the grid we will work with
yv, xv = np.meshgrid(grid_df.LAT.unique(), grid_df.LON.unique())
for i, place in shp_df.iterrows(): # Now we go through all the shapes
    # show which gridcell centers are contained inside the shape
    # ignore the warning caused by shapely using an old version of numpy
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        inplace = shapely.vectorized.contains(place.geometry, xv, yv)
    idx = np.argwhere(inplace)
    # Get the number of cells contained in the shape
    number_cells = idx.size/2
    if number_cells == 0:
        # If we have no cell centers in the shape, get the shape center and the cell which contains it
        c = place.geometry.centroid
        lon = c.x//degrees*degrees+degrees*0.5
        lat = c.y//degrees*degrees+degrees*0.5
        da_df = grid_df[(grid_df['LON']==lon) & (grid_df['LAT']==lat)]
        shp_grid.append({"shpfile_id": i, "grid_df_id": da_df.index[0]})
    else:
        for point in idx:
            lon = grid_df.LON.unique()[point[0]]
            lat = grid_df.LAT.unique()[point[1]]
            da_df = grid_df[(grid_df['LON']==lon) & (grid_df['LAT']==lat)]
            shp_grid.append({"shpfile_id": i, "grid_df_id": da_df.index[0]}) 

shp_grid_df = pd.DataFrame.from_dict(shp_grid)
    

shp_grid_df.head()

,shpfile_id,grid_df_id
0,0,4114
1,0,3971
2,0,4115
3,0,3972
4,0,4116


In [7]:
feature_mapping = {
    "ADM1": ["Admin-1 scale rank", "Admin-1 aggregation", "Admin-1 minor island"],
    "PCLI": ["Admin-0 country"],
    'MTS': ['Range/mtn'],
    'PLAT': ['Plateau'],
    'PLN': ['Plain'],
    'DSRT': ['Desert'],
    'OCN': ['ocean'],
    'SEA': ['sea', 'bay'],
    'GULF': ['gulf', 'bay'],
    'BAY': ['gulf', 'bay'],
    'CHN': ['channel'],
    'BSNU': ['basin']
}

In [8]:
# We add all the other codes we don't cover with blank shapefile classes
for fcode in places.feature_code.unique():
    if fcode not in feature_mapping:
        feature_mapping[fcode] = []

In [9]:
# To help match, we will rename some shapes so that they are the same as the name in our database
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Altay", case=False)),"name"] = "Altay"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Appalach", case=False)),"name"] = "Appalachian Mountains"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("cant", case=False)),"name"] = "Cordillera Cantábrica"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Dabie", case=False)),"name"] = "Dabie Shan"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("EASTERN GHATS", case=False)),"name"] = "Eastern Ghāts"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("WESTERN GHATS", case=False)),"name"] = "Western Ghāts"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("kunlun", case=False)),"name"] = "Kalakunlun Shan"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("LEN MOUNTAIN", case=False)),"name"] = "Kölen"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Taihang Mts.", case=False)),"name"] = "Taihang Shan"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Tatra Mts.", case=False)),"name"] = "Tatry"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("TIAN SHAN", case=False)),"name"] = "Tien Shan"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("andes", case=False)),"name"] = "Andes Mountains"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("HINDU KUSH", case=False)),"name"] = "Hindū Kush"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Marrah Mts", case=False)),"name"] = "Jabal Marrah"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("Lebanon", case=False)),"name"] = "Mount Lebanon"
shp_df.loc[(shp_df["featurecla"]=="Range/mtn") & (shp_df["name"].str.contains("KARAKORAM RA", case=False)),"name"] = "Karakorum Shan"

shp_df.loc[(shp_df["featurecla"]=="Desert") & (shp_df["name"].str.contains("Negev", case=False)), "name"] = "Negev"
shp_df.loc[(shp_df["featurecla"]=="Desert") & (shp_df["name"].str.contains("Atacama", case=False)), "name"] = "Atacama Desert"
shp_df.loc[(shp_df["featurecla"]=="Desert") & (shp_df["name"].str.contains("CHIHUAHUAN DESERT", case=False)), "name"] = "Chihuahua Desert"
shp_df.loc[(shp_df["featurecla"]=="Desert") & (shp_df["name"].str.contains("Lut desert", case=False)), "name"] = "God-e Lut"
shp_df.loc[(shp_df["featurecla"]=="Desert") & (shp_df["name"].str.contains("TAKLIMAKAN DESERT", case=False)), "name"] = "Takla Makan Desert"

shp_df.loc[
    (shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("cumberland", case=False)),["name","featurecla"]
] = ["Cumberland Plateau", "Plain"]
shp_df.loc[
    (shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("colorado", case=False)),["name","featurecla"]
] = ["San Francisco Plateau", "Plain"]
shp_df.loc[(shp_df["featurecla"]=="Plain") & (shp_df["name"].str.contains("gange", case=False)),"name"] = "Gangetic Plain"
shp_df.loc[(shp_df["featurecla"]=="Plain") & (shp_df["name"].str.contains("north china", case=False)),"name"] = "Huanghuai Pingyuan"

shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("mongol", case=False)), "name"] = "Nei Mongol Gaoyuan"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("deccan", case=False)), "name"] = "Deccan"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("chota", case=False)), "name"] = "Chota Nāgpur Plateau"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("loess", case=False)), "name"] = "Huangtu Gaoyuan"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("khorat", case=False)), "name"] = "Khorat Plateau"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("tibet", case=False)), "name"] = "Qing Zang Gaoyuan"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("polar", case=False)), "name"] = "South Polar Plateau"
shp_df.loc[(shp_df["featurecla"]=="Plateau") & (shp_df["name"].str.contains("YUNGUI", case=False)), "name"] = "Yungui Gaoyuan"

places["place_name"] = places["place_name"].str.lower().str.replace("mts.","mountains") 
shp_df["name"] = shp_df["name"].str.lower().str.replace("mts.","mountains") 

/tmp/ipykernel_375273/2263670162.py:43: FutureWarning: The default value of regex will change from True to False in a future version.
  places["place_name"] = places["place_name"].str.lower().str.replace("mts.","mountains")
/tmp/ipykernel_375273/2263670162.py:44: FutureWarning: The default value of regex will change from True to False in a future version.
  shp_df["name"] = shp_df["name"].str.lower().str.replace("mts.","mountains")


In [ ]:
# We'll start off with an empty dataframe
shp_df_matches = pd.DataFrame()

# And define the grid box size
degrees = 2.5

# Now we want to look through the feature type dictionary we had before
for place_key, shpfile_keys in feature_mapping.items():
    
    print(place_key)
    
    # We can get all the places and all the shapes
    feature_places = places.loc[places["feature_code"]==place_key]
    feature_shapes = shp_df.loc[shp_df["featurecla"].isin(shpfile_keys)]
    
    # And loop through the places
    for place, group in feature_places.groupby("place_name"):
        # If we don't have a shapefile feature type we just take the grid cell containing the point
        if not shpfile_keys:
            shp_id = None # we set shp_id to None, round the coordinates, and take the gridcell which has these coordinates
            lon = group.lon.values[0]//degrees*degrees+degrees*0.5
            lat = group.lat.values[0]//degrees*degrees+degrees*0.5
            grid_df_ids = grid_df[(grid_df['LON']==lon) & (grid_df['LAT']==lat)].index

        else:
            # Otherwise, we are going to try to find the places which match
            place_shapes = pd.DataFrame()
            # First we will try by geoname ids, if we have these
            if feature_shapes["gn_id"].sum() > 0:
                place_shapes = feature_shapes.loc[
                    (feature_shapes["gn_id"]==group.geonameid.values[0])
                ]
            # Then we try by country code, if we are dealing with a country
            if place_shapes.shape[0]==0 and place_key=="PCLI":
                place_shapes = feature_shapes.loc[
                    feature_shapes["ADM0_A3"] == group.country_predicted.values[0]
                ]
            # Then we try by name
            if place_shapes.shape[0]==0:       
                place_shapes = feature_shapes.loc[
                    (feature_shapes.name == place)
                ]    

            if place_shapes.shape[0]==0:
                continue # These are the places we cannot match, we need to develop other strategies for these, e.g. using other shapefile libraries
    #            if key=="ADM1": # can also try using gadm data, which we need to download separately
    #                 place_shapes = adm1shps_alt[
    #                     (adm1shps_alt["NAME_1"]==group.place_name.values[0]) |
    #                     (adm1shps_alt["VARNAME_1"].str.contains(group.place_name.values[0]))
    #                 ]
            else:
                shp_id = place_shapes.index[0]
                grid_df_ids = shp_grid_df.loc[
                    (shp_grid_df["shpfile_id"]==shp_id),
                    "grid_df_id"
                ]
        # For each document with this place, we add a row for each grid cell index matching the place
        for did in group.doc_id.unique():
            shp_df_matches = pd.concat([
                shp_df_matches,
                pd.DataFrame.from_dict({
                    "grid_df_id": grid_df_ids,
                    "doc_id": [did] * len(grid_df_ids),
                    "shp_id": [shp_id] * len(grid_df_ids),
                    "place": place
                })
            ])
            
            
print(shp_df_matches.shape)
shp_df_matches.head()

ADM1
PCLI


In [ ]:
shp_df_matches.to_csv("data/shp_df_matches.csv",index=False)

In [ ]:
import rioxarray

rds = rioxarray.open_rasterio("data/gpw_v4_population_count_rev11_2020_1_deg.asc",
)

rds = rds.squeeze().drop("spatial_ref").drop("band")
rds.name = "population"
pop_df = rds.to_dataframe().reset_index()

pop_df['LAT'] = pop_df['y']//2.5*2.5+1.25
pop_df['LON'] = pop_df['x']//2.5*2.5+1.25
pop_df.loc[pop_df['population']==-9999,"population"]=np.NaN
pop_df = pop_df.groupby(["LAT","LON"])['population'].sum().reset_index()
pop_df

In [ ]:
import math
def area_cell(lat, lon, degrees): 
    # calculate the area of a gridcell given the center lat and lon and the size in degrees
    if lon <0:
        lon+=360
    R = 6371
    f0 = math.radians(lat-degrees*0.5)
    f1 = math.radians(lat+degrees*0.5)
    l0 = math.radians(lon-degrees*0.5)
    l1 = math.radians(lon+degrees*0.5)

    return (math.sin(f1)-math.sin(f0)) * (l1 - l0) * R**2


grid_df['area'] = grid_df.apply(lambda x: area_cell(x['LAT'], x['LON'], 2.5), axis=1)

In [ ]:
np.linspace(0,2.5,5)

In [ ]:
land_masks = []
n = 5
land_array = np.empty((grid_df.shape[0], n*n))
i = 0
for x in np.linspace(0,2.5,n):
    for y in np.linspace(0,2.5,n):
        land_array[:,i] = globe.is_land(grid_df.LAT+y-1.25, grid_df.LON+x-1.25)
        i+=1
        
land_mask = land_array.sum(axis=1)>0

In [ ]:
grid_df['is_land'] = land_mask
grid_df.loc[grid_df['LAT']<-60,'is_land'] = False

In [ ]:
grid_df.is_land.sum()

In [ ]:
ncc_data = pd.read_csv("data/2_merged_da_data.csv")
grid_df = grid_df.merge(
    ncc_data[["LAT","LON","updated_precip","updated_temp"]].rename(columns={"updated_precip": "precip_da", "updated_temp": "temp_da"})
).merge(pop_df)
grid_df.reset_index().to_csv("data/grid_df.csv", index=False)
grid_df.head()